In [32]:
import pandas as pd
import plotly.express as px
import datetime
import sys

#hacky way to use sibling module 
# add ` "python.analysis.extraPaths": ["app/"] ` to .vscode/settings.json for pylance
sys.path.append('../app')
import myfitnesspal.analysis as mfpa
import myfitnesspal.diary_scraping as mfps

In [33]:
diary_df = pd.DataFrame()
for df in mfps.get_diary_data(datetime.date(2022,3,18), datetime.date(2022,3,22), user="ismailmo"):
    diary_df = pd.concat([diary_df, df], axis=0, join="outer")

In [34]:
macro_kcals = {'carbs_g':4, 'fat_g':9, 'protein_g':4}

#add kcal column for macros
for key,val in macro_kcals.items():
    diary_df[key.split('_')[0]+'_kcal'] = diary_df[key] * val

In [35]:
diary_df.columns

Index(['food', 'calories_kcal', 'carbs_g', 'fat_g', 'protein_g', 'sugar_g',
       'fiber_g', 'goal_calories_kcal', 'goal_carbs_g', 'goal_fat_g',
       'goal_protein_g', 'goal_sugar_g', 'goal_fiber_g', 'qty', 'date',
       'carbs_kcal', 'fat_kcal', 'protein_kcal'],
      dtype='object')

In [36]:
diary_df_kcals = diary_df[['food', 'calories_kcal', 'carbs_kcal',
       'fat_kcal', 'protein_kcal', 'date']]

In [37]:
diary_df_kcals.head()

,food,calories_kcal,carbs_kcal,fat_kcal,protein_kcal,date
0,The Cheese Emporium - Mature Lighter Cheese,96,4,63,32,2022-03-18
1,Hovis - 50/50 Bread,182,128,9,32,2022-03-18
2,chicken thigh cooked,259,0,81,168,2022-03-18
3,Cereals - Oats instant fortified plain dry (in...,148,104,18,24,2022-03-18
4,Blueberries,86,88,0,4,2022-03-18


In [38]:
melted_df = diary_df_kcals.drop("calories_kcal", axis=1).melt(['food', 'date'])
melted_df = melted_df.rename({"value":"kcal"}, axis=1)
melted_df['meals'] = 1

melted_df = melted_df.groupby(['food', 'date', 'variable']).sum()
melted_df= melted_df.reset_index()
melted_df[melted_df['date']==datetime.date(2022,3,21)]

,food,date,variable,kcal,meals
12,Babybel - Light,2022-03-21,carbs_kcal,0,1
13,Babybel - Light,2022-03-21,fat_kcal,54,1
14,Babybel - Light,2022-03-21,protein_kcal,48,1
39,Fibre One - Peanut Butter Popcorn Bar,2022-03-21,carbs_kcal,36,1
40,Fibre One - Peanut Butter Popcorn Bar,2022-03-21,fat_kcal,36,1
41,Fibre One - Peanut Butter Popcorn Bar,2022-03-21,protein_kcal,8,1
63,Kellogs - Rice Krispies Square - Chocolate,2022-03-21,carbs_kcal,216,2
64,Kellogs - Rice Krispies Square - Chocolate,2022-03-21,fat_kcal,90,2
65,Kellogs - Rice Krispies Square - Chocolate,2022-03-21,protein_kcal,16,2
105,chicken thigh cooked,2022-03-21,carbs_kcal,0,3


In [39]:

fig = px.treemap(melted_df, path=[px.Constant("all"),
'variable','food', 'date'], values='kcal', hover_data=['meals'])

fig.show()

/root/.local/lib/python3.10/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/root/.local/lib/python3.10/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/root/.local/lib/python3.10/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/root/.local/lib/python3.10/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [40]:
diary_df['date'] = pd.to_datetime(diary_df['date'])

In [73]:
macro_kcals = {'carbs':4, 'fat':9, 'protein':4}
macro_colours = {'carbs':"darkturquoise", 'fat':"tomato", 'protein':"mediumorchid"}
#add kcal column for macros
for key,val in macro_kcals.items():
    diary_df[key+'_kcal'] = diary_df[key+"_g"] * val
    diary_df["goal_" + key+'_kcal'] = diary_df["goal_" +key+"_g"] * val

In [74]:
sum_cols = {col:'sum' for col in ['calories_kcal', 'carbs_g', 'fat_g', 'protein_g','carbs_kcal', 'fat_kcal', 'protein_kcal']}

In [75]:
mean_cols = {col:'mean' for col in [ 'goal_calories_kcal', 'goal_carbs_g', 'goal_fat_g',
       'goal_protein_g', 'goal_carbs_kcal', 'goal_fat_kcal',
       'goal_protein_kcal']}

In [76]:
agg_dict = {**sum_cols, **mean_cols}

In [77]:
daily_data = diary_df.groupby('date').agg(agg_dict)

In [78]:
kcal_macro_cols = [col for col in daily_data.columns if 'kcal' in col and 'calories' not in col]
gram_macro_cols = [col for col in daily_data.columns if '_g' in col and 'calories' not in col]
calorie_cols = [col for col in daily_data.columns if 'calories' in col]

In [109]:
import plotly.graph_objects as go
fig = go.Figure()


for col in gram_macro_cols:
    macro = col.split("_")[-2]
    color = macro_colours[macro]
    if 'goal' in col:
        line={"dash":"dash", "color": color}
        fig.add_trace(go.Scatter(x=daily_data.index, 
        y=daily_data[col], 
        mode='lines', 
        name="goal", 
        line=line,        
        legendgroup=macro))
    else:
        fig.add_trace(go.Bar(x=daily_data.index, 
        y=daily_data[col], 
        name=macro,
        marker_color= color,
        legendgroup=macro,
        customdata =daily_data[macro+"_g"],
        hovertemplate='%{customdata}'))
fig.update_layout(title='Actual Intake vs Goals', xaxis_title ="Date", yaxis_title="Calories")
fig.show()

In [106]:
calorie_cols

['calories_kcal', 'goal_calories_kcal']

In [110]:
import numpy as np

In [143]:
daily_data[calorie_cols].reset_index()

,date,calories_kcal,goal_calories_kcal
0,2022-03-18,1779,1800.0
1,2022-03-19,1897,1800.0
2,2022-03-20,2235,1800.0
3,2022-03-21,1480,1800.0
4,2022-03-22,1745,1800.0


In [155]:
daily_data['calories_kcal'].apply(lambda x:list(zip(x,daily_data['calories_kcal'])))

TypeError: 'int' object is not iterable

In [157]:
pd.Series((zip(daily_data['calories_kcal'],daily_data['calories_kcal'])))

0    (1779, 1779)
1    (1897, 1897)
2    (2235, 2235)
3    (1480, 1480)
4    (1745, 1745)
dtype: object

In [164]:
np.vstack([daily_data['calories_kcal'],daily_data['calories_kcal']])

array([[1779, 1897, 2235, 1480, 1745],
       [1779, 1897, 2235, 1480, 1745]])